In [1]:
import numpy as np
import pandas as pd
import pickle

from keras import models
from keras import layers
from keras import optimizers
from keras import losses
from keras import activations
from keras import metrics
from keras import regularizers
import math

import random

# to make the experimens replicable
random.seed(123456)

Using TensorFlow backend.


## Important

as first step run the notebook `1. classification-dataset-creation.ipynb` to create the dataset.

In [2]:
dataset_path = '../data/dataset-dirigenti-split.pkl'

with open(dataset_path, 'rb') as f:
    train_samples, train_labels, val_samples, val_labels, test_samples, test_labels = pickle.load(f)

### Transform it in a dataset

In [3]:
train_samples[:5]

array([ 'RESIDUI PERENTI BILANCIO 1997 - IMPEGNO E LIQUIDAZIONE ALL ORDINE PROVINCIALE DEI MEDICI-CHIRURGHI E DEGLI ODONTOIATRI DI FIRENZE PERL ATTUAZIONE DI INIZIATIVE CULTURALI E SCIENTIFICHE DI SUPPORTO AI CORSI DI FORMAZIONE SPECIFICA IN MEDICINA GENERALE.',
       'L.R. 72/94 " Danni causati da animali predatori o da eventi meteorici al patrimonio zootecnico" - Liquidazione indennizzi beneficiari Euro 34.988,28=.',
       'Eventi alluvionali novembre 2000 - Interventi urgenti connessi alla difesa idraulica - Erogazione al Comune di Montemurlo (PO) saldo finanziamento spettante per realizzazione intervento n. 301 compreso nel Piano regionale di cui all`allegato 2 della DGRT n. 104/2003.',
       'APPROVAZIONE SCHEMA DI CONVENZIONE TRA LA REGIONE TOSCANA E I SOGGETTI PRIVATI AMMESSI A CONTRIBUTO PER I PROGETTI-INTERVENTO IN MATERIA DI PAESAGGIO',
       'Contributo al Comune di Siena per l`organizzazione della Terza giornata del Contemporaneo in programma nel mese di ottobre 2007.']

In [4]:
train_labels[:5]

array(['01943', '01934',
       'DIREZIONE GENERALE POLITICHE TERRITORIALI E AMBIENTALI',
       'DIREZIONE GENERALE POLITICHE TERRITORIALI E AMBIENTALI',
       'DIREZIONE GENERALE POLITICHE FORMATIVE, BENI E ATTIVIT'],
      dtype='<U54')

In [5]:
samples = np.concatenate([train_samples,val_samples, test_samples])
samples.shape

(138043,)

In [6]:
labels = np.concatenate([train_labels, val_labels, test_labels])
labels.shape

(138043,)

## Tokenize the Dataset

In [7]:
from keras.preprocessing.text import Tokenizer

In [8]:
num_words = 10000

In [9]:
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(samples)

In [10]:
freq_words = [(v,k) for k,v in tokenizer.word_docs.items()]

In [11]:
sorted(freq_words, key=lambda t: t[0], reverse=True)[:10]

[(108923, 'di'),
 (70047, 'e'),
 (62030, 'del'),
 (60358, 'per'),
 (48081, 'r'),
 (46824, 'l'),
 (44301, 'n'),
 (43630, 'a'),
 (40125, 'della'),
 (32549, 'in')]

In [12]:
train_sequences = tokenizer.texts_to_sequences(train_samples)
val_sequences = tokenizer.texts_to_sequences(val_samples)
test_sequences = tokenizer.texts_to_sequences(test_samples)

In [13]:
for s in train_sequences[:2]:
    print(s) 

[1120, 915, 418, 721, 14, 2, 18, 50, 2055, 530, 21, 1072, 2, 52, 1, 66, 179, 1, 691, 749, 2, 5399, 1, 512, 25, 723, 1, 129, 1100, 10, 558, 321]
[7, 6, 956, 88, 740, 2101, 45, 1187, 3733, 132, 45, 277, 4270, 13, 313, 1613, 18, 2539, 965, 221, 708, 7629, 298]


Convert labels to sequences

In [14]:
index_to_labels = dict(enumerate(set(labels)))
labels_to_index = {v:k for k,v in index_to_labels.items()}

In [15]:
len(index_to_labels)

20

In [16]:
print(labels_to_index)
print(len(labels_to_index))

{'D.G.  AVVOCATURA                                      ': 0, 'DIREZIONE GENERALE POLITICHE FORMATIVE, BENI E ATTIVIT': 1, 'DIREZIONE GENERALE POLITICHE TERRITORIALI E AMBIENTALI': 2, '01946': 3, '01934': 4, 'DIREZIONE ISTRUZIONE E FORMAZIONE': 5, "D.G. COMPETITIVITA' DEL SISTEMA REGIONALE E SVILUPPO D": 6, 'POLITICHE AMBIENTALI, ENERGIA E CAMBIAMENTI CLIMATICI': 7, 'DIREZIONE GENERALE DIRITTO ALLA SALUTE E POLITICHE DI ': 8, '01937': 9, 'DIREZIONE ORGANIZZAZIONE E SISTEMI INFORMATIVI': 10, '01928': 11, 'DIREZIONE GENERALE SVILUPPO ECONOMICO                 ': 12, 'DIREZIONE AGRICOLTURA E SVILUPPO RURALE': 13, 'DIREZIONE DIFESA DEL SUOLO E PROTEZIONE CIVILE': 14, 'DIREZIONE GENERALE BILANCIO E FINANZE                 ': 15, 'DIREZIONE DIRITTI DI CITTADINANZA E COESIONE SOCIALE': 16, '01025': 17, 'D.G. PRESIDENZA                                       ': 18, '01943': 19}
20


In [17]:
encoded_train_labels = [labels_to_index[l] for l in train_labels]
encoded_val_labels = [labels_to_index[l] for l in val_labels]
encoded_test_labels = [labels_to_index[l] for l in test_labels]

In [18]:
encoded_train_labels[:10]

[19, 4, 2, 2, 1, 0, 10, 5, 15, 6]

### Preparing Data

- for the input data we pad the sequence to a max length in order to make uniform the sequences.
- for the labels we continue with one-hot encoding


In [19]:
from keras.preprocessing import sequence

In [20]:
print('max length of objects {}'.format(max(map(len, train_sequences))))

max length of objects 150


#### Maximum Sequence Length

for initial test we use the first 100 words

In [21]:
maxlen = 100

In [22]:
x_train_seq = sequence.pad_sequences(train_sequences, maxlen=maxlen)
x_val_seq = sequence.pad_sequences(val_sequences, maxlen=maxlen)
x_test_seq = sequence.pad_sequences(test_sequences, maxlen=maxlen)

In [23]:
x_train_seq.shape

(99390, 100)

One-hot encoding for input

In [24]:
def vectorize_sequences(sequences, dimension=10000):
    data = np.zeros((len(sequences), dimension), dtype=np.float32)
    for i, sequence in enumerate(sequences):
        data[i,sequence] = 1.
    return data

In [25]:
x_train = vectorize_sequences(train_sequences, num_words)
x_val = vectorize_sequences(val_sequences, num_words)
x_test = vectorize_sequences(test_sequences, num_words)

One hot encoding for the labels

In [26]:
def to_one_hot(labels):
    results = np.zeros((len(labels), len(set(labels))), dtype=np.float32)
    for i, label in enumerate(labels):
        results[i, label] = 1.
    return results

In [27]:
y_train = to_one_hot(encoded_train_labels)
y_val = to_one_hot(encoded_val_labels)
y_test = to_one_hot(encoded_test_labels)

In [28]:
y_train

array([[ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.]], dtype=float32)

## The Effects of Word Embeddings

Before using a word embedding as a layer in our network let evaluate the effect of embeddings by training a simple classifier that has it as only layer.

In [29]:
def build_model_only_embeddings(embed_size):
    model = models.Sequential()
    model.add(layers.Embedding(input_dim=num_words, output_dim=embed_size, input_length=maxlen, name='embed'))
    model.add(layers.Flatten())
    model.add(layers.Dense(len(labels_to_index), activation='softmax', name='softmax'))
    model.compile(optimizer=optimizers.Adam(), 
          loss='categorical_crossentropy',
          metrics=['accuracy'])
    return model

In [ ]:
model_only_embed_small = build_model_only_embeddings(32)
model_only_embed_medium = build_model_only_embeddings(64)
model_only_embed_large = build_model_only_embeddings(128)

In [ ]:
history_embed_small = model_only_embed_small.fit(x=x_train_seq, y=y_train, validation_data=(x_val_seq, y_val),
                   epochs=10, batch_size=256)

In [ ]:
history_embed_medium = model_only_embed_medium.fit(x=x_train_seq, y=y_train, validation_data=(x_val_seq, y_val),
                   epochs=10, batch_size=256)

In [ ]:
history_embed_large = model_only_embed_large.fit(x=x_train_seq, y=y_train, validation_data=(x_val_seq, y_val),
                   epochs=10, batch_size=256)

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
def chart_loss(history):
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(1, len(loss) + 1)

    plt.plot(epochs, loss, 'b+', label='Training Loss')
    plt.plot(epochs, val_loss, 'b', label='Validation Loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.yticks(np.arange(0,2.2, step=0.2))
    plt.xticks(epochs)
    plt.legend()
    plt.show()

In [ ]:
def chart_acc(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']

    epochs = range(1, len(acc) + 1)

    plt.plot(epochs, acc, 'b+', label='Training Acc')
    plt.plot(epochs, val_acc, 'b', label='Validation Acc')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Accuray')
    plt.yticks(np.arange(0.4,1.05, step=0.05))
    plt.xticks(epochs)
    plt.legend()
    plt.show()

In [ ]:
chart_loss(history_embed_small)
chart_loss(history_embed_medium)
chart_loss(history_embed_large)

We can see how the model with larger embedding size tend to overfit faster than the others (respectively epochs 2, 3, and 4). However, from a loss point of view all the models starts overfitting with a loss values ~ 0.8.
Let's now analyze values for accuracy.

In [ ]:
chart_acc(history_embed_small)
chart_acc(history_embed_medium)
chart_acc(history_embed_large)

In [ ]:
def min_loss(history):
    val_loss = history.history['val_loss'] 
    return np.argmin(val_loss) + 1 

def accuracy(history, epoch):
    val_acc = history.history['val_acc']
    return val_acc[epoch - 1]

In [ ]:
print('min loss for model small is {}'.format(min_loss(history_embed_small)))
print('min loss for model medium is {}'.format(min_loss(history_embed_medium)))
print('min loss for model large is {}'.format(min_loss(history_embed_large)))

In [ ]:
print('best validation accuracy for model small is {}'.format(
    accuracy(history_embed_small, min_loss(history_embed_small))))
print('best validation accuracy for model medium is {}'.format(
    accuracy(history_embed_medium, min_loss(history_embed_medium))))
print('best validation accuracy for model large is {}'.format(
    accuracy(history_embed_large, min_loss(history_embed_large))))

The results show how only using an flat classifier with an embedding layer we obtain an accuracy ~ 83%. Which is a pretty good result.
The next steps is use an embedding layers in a Feed Forward Neural Network.

## Build the Models

We take into account:
- a base model that uses dropout
- a embed model that adds an embedding layer

In [ ]:
def build_model_dropout(neurons):
    model = models.Sequential()
    model.add(layers.Dense(neurons, activation='relu', input_shape=(x_train.shape[-1], )))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(neurons, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(neurons, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(len(labels_to_index), activation='softmax'))
    
    model.compile(optimizer=optimizers.Adam(), 
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    return model

In [ ]:
def build_model_embeddings(neurons, embed_size):
    model = models.Sequential()
    model.add(layers.Embedding(input_dim=num_words, output_dim=embed_size, input_length=maxlen))
    model.add(layers.Flatten())
    model.add(layers.Dense(neurons, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(neurons, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(len(labels_to_index), activation='softmax'))
    model.compile(optimizer=optimizers.Adam(), 
          loss='categorical_crossentropy',
          metrics=['accuracy'])
    return model

In [ ]:
model_dropout = build_model_dropout(256)

In [ ]:
history_dropout = model_dropout.fit(x=x_train, y=y_train, validation_data=(x_val, y_val),
                   epochs=10, batch_size=256)

In [ ]:
model_embed = build_model_embeddings(128,64)

In [ ]:
history_embed = model_embed.fit(x=x_train_seq, y=y_train, validation_data=(x_val_seq, y_val),
                   epochs=10, batch_size=256)

### Printing The Loss

In [ ]:
chart_loss(history_embed)
chart_loss(history_dropout)

From the charts we can see that: 
- word embedding seems to not improve the accuracy of the model. This is usual in the case you have little training data available. From another point of view it is surprising to see that the classifier with only an embedding layer perform better than the one taken into account here
- the model with dropout perform better than the model with embeddings.

From this result one could gather that embeddings are not useful. This is wrong, because we didn't consider that we changed another thing with the current represantion. The model that uses embedding takes as input sequence of ids and not a vectorize representation of subjects. 

In the next post we will explore the usage of LSTM to address this kind of representation. 

In [ ]:
chart_acc(history_embed)
chart_acc(history_dropout)

From the chart above we can see that the best model is the model that uses dropout, while the best epoch is the 6 where the validation accuracy crosses the training accuracy

In [ ]:
def compare_loss(histories):
    epochs = range(1, len(list(histories.values())[0].history['val_loss']) + 1)

    for i, history in histories.items():
        val_loss = history.history['val_loss']
        plt.plot(epochs, val_loss, label='Validation Loss {}'.format(i))
            
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

In [ ]:
compare_loss({'embedding': history_embed, 'Dropout': history_dropout})

Another interesting factor is that the loss of the model that uses embeddings grows during the training. It shows that embedding trained with the model aren't useful.
Another approach can be uses:
- pretrained embeddings
- remove some of the most frequent words

In [ ]:
def compare_accuracy(histories):
    epochs = range(1, len(list(histories.values())[0].history['val_acc']) + 1)

    for i, history in histories.items():
        val_loss = history.history['val_acc']
        plt.plot(epochs, val_loss, label='Validation Accuracy {}'.format(i))
            
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

In [ ]:
compare_accuracy({'embedding': history_embed, 'Dropout': history_dropout})

In [ ]:
print('min loss for model Embed {}'.format(min_loss(history_embed)))
print('min loss for model Dropout is {}'.format(min_loss(history_dropout)))

In [ ]:
print('best validation accuracy for model Embed {}'.format(
    accuracy(history_embed, min_loss(history_embed))))
print('best validation accuracy for model Dropout {}'.format(
    accuracy(history_dropout, min_loss(history_dropout))))

### Evaluate on the Test Set

We still want to evaluate the models with embedding to understand if it overfits on input data.

In [ ]:
model = build_model_embeddings(128,64)

In [ ]:
history = model.fit(x=np.concatenate([x_train_seq, x_val_seq]), y=np.concatenate([y_train, y_val]), epochs=4, batch_size=256)

In [ ]:
loss, acc = model.evaluate(x_test_seq, y_test)

In [ ]:
print('loss {}'.format(loss))
print('acc {}'.format(acc))

## Conclusion

We evaluated how to use Embeddings as alternative to one-hot encoding. 
